In [ ]:
pip install transformers torch


In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_chat(chat_messages):
    # Join the chat messages into a single string
    chat_text = ' '.join(chat_messages)
    return chat_text


In [ ]:
def generate_mcqs(chat_text):
    # For simplicity, we will generate a few MCQs manually
    questions = [
        "What is the main topic of the conversation?",
        "Who mentioned the key point in the chat?",
        "What was the conclusion of the discussion?"
    ]

    mcqs = []
    for question in questions:
        inputs = tokenizer(question, chat_text, return_tensors='pt')
        outputs = model(**inputs)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

        # Get the most likely beginning and end of the answer
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

        # Generate distractors (simple heuristic for demonstration)
        distractors = [
            "Option 1",
            "Option 2",
            "Option 3",
            "Option 4"
        ]

        mcqs.append({
            "question": question,
            "correct_answer": answer,
            "distractors": distractors
        })

    return mcqs


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

def generate_article(chat_text):
    # Generate a summary of the chat
    summary = summarizer(chat_text, max_length=150, min_length=100, do_sample=False)
    article = summary[0]['summary_text']
    return article


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
def evaluate_mcqs(mcqs, chat_text):
    evaluation = []
    for mcq in mcqs:
        question = mcq["question"]
        correct_answer = mcq["correct_answer"]
        distractors = mcq["distractors"]

        # Evaluate relevance
        relevance = correct_answer in chat_text

        # Evaluate difficulty level (simple heuristic)
        difficulty_level = "moderate"  # This can be enhanced with more sophisticated methods

        # Evaluate correctness of options
        correctness_of_options = correct_answer not in distractors

        evaluation.append({
            "question": question,
            "relevance": relevance,
            "difficulty_level": difficulty_level,
            "correctness_of_options": correctness_of_options
        })

    return evaluation

def evaluate_article(article, chat_text):
    # Evaluate coherence (simple heuristic)
    coherence = "The article flows well with a clear introduction, body, and conclusion."

    # Evaluate grammar and style (simple heuristic)
    grammar_and_style = "The article maintains proper grammar and a formal or semi-formal tone."

    # Evaluate relevance
    relevance = any(word in chat_text for word in article.split())

    evaluation = {
        "coherence": coherence,
        "grammar_and_style": grammar_and_style,
        "relevance": relevance
    }

    return evaluation

In [ ]:
def main(chat_messages):
    chat_text = preprocess_chat(chat_messages)

    # Generate MCQs
    mcqs = generate_mcqs(chat_text)
    print("Generated MCQs:")
    for mcq in mcqs:
        print(f"Question: {mcq['question']}")
        print(f"Correct Answer: {mcq['correct_answer']}")
        print(f"Distractors: {', '.join(mcq['distractors'])}")
        print()

    # Evaluate MCQs
    mcq_evaluation = evaluate_mcqs(mcqs, chat_text)
    print("MCQ Evaluation:")
    for eval in mcq_evaluation:
        print(f"Question: {eval['question']}")
        print(f"Relevance: {eval['relevance']}")
        print(f"Difficulty Level: {eval['difficulty_level']}")
        print(f"Correctness of Options: {eval['correctness_of_options']}")
        print()

    # Generate Article
    article = generate_article(chat_text)
    print("Generated Article:")
    print(article)

    # Evaluate Article
    article_evaluation = evaluate_article(article, chat_text)
    print("Article Evaluation:")
    print(f"Coherence: {article_evaluation['coherence']}")
    print(f"Grammar and Style: {article_evaluation['grammar_and_style']}")
    print(f"Relevance: {article_evaluation['relevance']}")

In [ ]:
chat_messages = [
    "Alice: What is the capital of France?",
    "Bob: The capital of France is Paris.",
    "Alice: Thank you!",
    "Bob: You're welcome.",
    "Alice: Do you know the population of Paris?",
    "Bob: The population of Paris is around 2.2 million.",
    "Alice: That's interesting.",
    "Bob: Yes, it's a beautiful city.",
    "Alice: I would love to visit someday.",
    "Bob: You should! It's worth it.",
    "Alice: What are some famous landmarks in Paris?",
    "Bob: The Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral are some famous landmarks.",
    "Alice: I've heard the Eiffel Tower is amazing.",
    "Bob: It is! The view from the top is breathtaking.",
    "Alice: How tall is the Eiffel Tower?",
    "Bob: The Eiffel Tower is about 324 meters tall.",
    "Alice: Wow, that's impressive.",
    "Bob: Indeed, it's one of the most iconic structures in the world.",
    "Alice: What about the Louvre Museum?",
    "Bob: The Louvre is the world's largest and most visited art museum.",
    "Alice: What are some famous paintings there?",
    "Bob: The Mona Lisa by Leonardo da Vinci is one of the most famous.",
    "Alice: I've always wanted to see the Mona Lisa.",
    "Bob: It's a must-see if you visit Paris.",
    "Alice: What other artworks are famous there?",
    "Bob: The Winged Victory of Samothrace and the Venus de Milo are also very famous.",
    "Alice: Those sound amazing.",
    "Bob: They are! The Louvre has a vast collection of art from different periods.",
    "Alice: How about the Notre-Dame Cathedral?",
    "Bob: Notre-Dame is a historic Catholic cathedral known for its French Gothic architecture.",
    "Alice: I heard there was a fire there recently.",
    "Bob: Yes, there was a devastating fire in 2019, but restoration efforts are underway.",
    "Alice: That's good to know.",
    "Bob: Paris has so much history and culture to offer.",
    "Alice: I can't wait to visit.",
    "Bob: You'll have a fantastic time!",
    "Alice: Thanks for all the information.",
    "Bob: You're welcome! Have a great day."
]


In [ ]:
main(chat_messages)

Generated MCQs:
Question: What is the main topic of the conversation?
Correct Answer: 
Distractors: Option 1, Option 2, Option 3, Option 4

Question: Who mentioned the key point in the chat?
Correct Answer: 
Distractors: Option 1, Option 2, Option 3, Option 4

Question: What was the conclusion of the discussion?
Correct Answer: 
Distractors: Option 1, Option 2, Option 3, Option 4

MCQ Evaluation:
Question: What is the main topic of the conversation?
Relevance: True
Difficulty Level: moderate
Correctness of Options: True

Question: Who mentioned the key point in the chat?
Relevance: True
Difficulty Level: moderate
Correctness of Options: True

Question: What was the conclusion of the discussion?
Relevance: True
Difficulty Level: moderate
Correctness of Options: True

Generated Article:
 The population of Paris is around 2.2 million, says Bob . The Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral are some famous landmarks . The Mona Lisa by Leonardo da Vinci is one of the most f